In [1]:
# from qiskit.opflow import I, Z, X, Y, SparsePauliOp
import numpy as np

# 假设有 3 台机器和 3 个操作
m = 3  # 机器数量
n = 3  # 操作数量
l = 4  # 用于表示 C_max 的二进制位数

# 处理时间矩阵 (例如)
p = np.array([[1, 2, 3],
              [2, 1, 2],
              [3, 2, 1]])

# 常数
K1 = 10
K2 = 10

# 量子比特数量
num_qubits = m * n + l + m * l


def pauli_z_term(index, num_qubits):
    """Creates a Pauli-Z term for the given index."""
    paulis = ['I'] * num_qubits
    paulis[index] = 'Z'
    return ''.join(paulis)


z_terms = []
for h in range(l):
    coef = 2 ** h / 2
    z_term = pauli_z_term(m * n + h, num_qubits)
    z_terms.append((coef, z_term))

constraint1_terms = []
for i in range(n):
    for j in range(m):
        coef = K1 / 2
        x_term = pauli_z_term(i * m + j, num_qubits)
        constraint1_terms.append((coef, x_term))

# 添加常数项和交叉项
constant_term = K1 * (m / 2 - 1) ** 2
cross_term_coef = -K1 * (m / 2 - 1)
cross_terms = [(cross_term_coef, pauli_z_term(i * m + j, num_qubits)) for i in range(n) for j in range(m)]

constraint2_terms = []
for j in range(m):
    for i in range(n):
        coef = K2 * p[i, j] / 2
        x_term = pauli_z_term(i * m + j, num_qubits)
        constraint2_terms.append((coef, x_term))

    for h in range(l):
        coef = K2 * 2 ** h / 2
        tau_term = pauli_z_term(m * n + j * l + h, num_qubits)
        z_term = pauli_z_term(m * n + h, num_qubits)
        constraint2_terms.append((coef, tau_term))
        constraint2_terms.append((-coef, z_term))

# 添加常数项和交叉项
constant_term2 = K2 * np.sum(p / 2)
cross_term_coef2 = -K2 * np.sum(p / 2)
cross_terms2 = [(cross_term_coef2, pauli_z_term(i * m + j, num_qubits)) for i in range(n) for j in range(m)]

from qiskit.opflow import PauliSumOp

# 构建哈密顿量
hamiltonian_terms = z_terms + constraint1_terms + cross_terms + cross_terms + constraint2_terms + cross_terms2

# 转换为 SparsePauliOp
pauli_strings = [term[1] for term in hamiltonian_terms]
coefficients = [term[0] for term in hamiltonian_terms]

hamiltonian = PauliSumOp.from_list(list(zip(pauli_strings, coefficients)))

print(hamiltonian)


0.5 * IIIIIIIIIZIIIIIIIIIIIIIII
+ 1.0 * IIIIIIIIIIZIIIIIIIIIIIIII
+ 2.0 * IIIIIIIIIIIZIIIIIIIIIIIII
+ 4.0 * IIIIIIIIIIIIZIIIIIIIIIIII
+ 5.0 * ZIIIIIIIIIIIIIIIIIIIIIIII
+ 5.0 * IZIIIIIIIIIIIIIIIIIIIIIII
+ 5.0 * IIZIIIIIIIIIIIIIIIIIIIIII
+ 5.0 * IIIZIIIIIIIIIIIIIIIIIIIII
+ 5.0 * IIIIZIIIIIIIIIIIIIIIIIIII
+ 5.0 * IIIIIZIIIIIIIIIIIIIIIIIII
+ 5.0 * IIIIIIZIIIIIIIIIIIIIIIIII
+ 5.0 * IIIIIIIZIIIIIIIIIIIIIIIII
+ 5.0 * IIIIIIIIZIIIIIIIIIIIIIIII
- 5.0 * ZIIIIIIIIIIIIIIIIIIIIIIII
- 5.0 * IZIIIIIIIIIIIIIIIIIIIIIII
- 5.0 * IIZIIIIIIIIIIIIIIIIIIIIII
- 5.0 * IIIZIIIIIIIIIIIIIIIIIIIII
- 5.0 * IIIIZIIIIIIIIIIIIIIIIIIII
- 5.0 * IIIIIZIIIIIIIIIIIIIIIIIII
- 5.0 * IIIIIIZIIIIIIIIIIIIIIIIII
- 5.0 * IIIIIIIZIIIIIIIIIIIIIIIII
- 5.0 * IIIIIIIIZIIIIIIIIIIIIIIII
- 5.0 * ZIIIIIIIIIIIIIIIIIIIIIIII
- 5.0 * IZIIIIIIIIIIIIIIIIIIIIIII
- 5.0 * IIZIIIIIIIIIIIIIIIIIIIIII
- 5.0 * IIIZIIIIIIIIIIIIIIIIIIIII
- 5.0 * IIIIZIIIIIIIIIIIIIIIIIIII
- 5.0 * IIIIIZIIIIIIIIIIIIIIIIIII
- 5.0 * IIIIIIZIIIIIIIIIIIIIIIIII
- 5.0 * IIIIIIIZ

C:\Users\18330\AppData\Local\Temp\ipykernel_22232\184413158.py:66: DeprecationWarning: The ``qiskit.opflow`` module is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  from qiskit.opflow import PauliSumOp


In [3]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit.opflow import PauliSumOp

# 使用 NumPyEigensolver 计算基态能量
numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(hamiltonian)
# 获取基态能量
ref_value = result.eigenvalue.real
print(f"Reference value: {ref_value:.5f}")

Reference value: -1017.50000
